In [1]:
import numpy as np
import pandas as pd

from geopy.distance import vincenty
import datetime
import time

In [2]:
s = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])

In [3]:
s

a    1
b    2
c    3
d    4
dtype: int64

In [5]:
df = pd.read_csv('201809-citibike-tripdata.csv')

In [6]:
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877879,369,2018-09-30 00:27:25.9840,2018-09-30 00:33:35.7070,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3631.0,Crown St & Bedford Ave,40.666563,-73.956741,32976,Subscriber,1985,2
1877880,191,2018-09-30 00:30:30.1850,2018-09-30 00:33:42.1090,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3631.0,Crown St & Bedford Ave,40.666563,-73.956741,15595,Subscriber,1985,2
1877881,1442,2018-09-30 08:10:03.1790,2018-09-30 08:34:05.3870,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,471.0,Grand St & Havemeyer St,40.712868,-73.956981,28646,Subscriber,1981,1
1877882,453,2018-09-30 12:20:13.6830,2018-09-30 12:27:46.9140,3601.0,Sterling St & Bedford Ave,40.662706,-73.956912,3584.0,Eastern Pkwy & Franklin Ave,40.670777,-73.957680,34272,Subscriber,1986,1


# 1. Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов

In [7]:
df.shape

(1877884, 15)

# 2. Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака

In [47]:
df['tripduration'].apply(lambda x: x / 60).mean().round(2)

AttributeError: 'float' object has no attribute 'round'

# 3. Сколько поездок начались и закончились в той же самой станции?

In [9]:
df[df['start station name'] == df['end station name']].shape

(41364, 15)

# 4. Какой самый используемый байк(bikeid) в городе по количеству поездок?

In [10]:
df['bikeid'].value_counts()

33875    559
33772    549
34357    549
33868    533
33878    532
        ... 
34867      1
15834      1
32379      1
34431      1
29726      1
Name: bikeid, Length: 10892, dtype: int64

# 5. Найдите идентификатор велосипеда (bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных

In [11]:
df.groupby(['bikeid'])[['tripduration']].mean().sort_values(by='tripduration')

,tripduration
bikeid,
32379,64.000000
25612,72.000000
14761,182.666667
34896,217.200000
22368,224.111111
...,...
18041,103932.100000
19083,335620.142857
17431,394605.833333


# 6. Сколько строк, в которых отсутствуют данные о start station id?

In [12]:
df['start station id'].isnull().value_counts()

False    1877168
True         716
Name: start station id, dtype: int64

# 7. Какова средняя продолжительность поездки в минутах в зависимости от типа подписки c точностью до 2 знака?

In [13]:
df.groupby(['usertype']).agg({'tripduration': lambda x: round((sum(x) / len(x)) / 60, 2)})

,tripduration
usertype,
Customer,33.42
Subscriber,13.33


# 8. Найдите среднюю длину поездок в километрах с точностью до 2 знака, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id).

Hint: можно воспользоваться библиотекой geopy и взять расстояние vincenty(минимальное расстояние между точками)

In [70]:
citybike['distance_km'] = citybike.apply(lambda x: vincenty((x['start station latitude'], x['start station longitude']),
                                                            (x['end station latitude'], x['end station longitude'])).kilometers, axis=1)

print(citybike['distance_km'].mean())

TypeError: cannot convert the series to <class 'float'>

In [ ]:
citybike['end station name'].value_counts()[:5]

# 9. Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера включительно

In [ ]:
citybike['start_hour'] = citybike['starttime'].apply(lambda x: datetime.datetime.fromtimestamp(
                                        time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [44]:
citybike['end_hour'] = citybike['stoptime'].apply(lambda x: datetime.datetime.fromtimestamp(
            time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

,start station id,starttime
0,252.0,2018-09-01 00:00:05.2690
1,314.0,2018-09-01 00:00:11.2810
2,3142.0,2018-09-01 00:00:20.6490
3,308.0,2018-09-01 00:00:21.7460
4,345.0,2018-09-01 00:00:27.3150
...,...,...
1877879,3601.0,2018-09-30 00:27:25.9840
1877880,3601.0,2018-09-30 00:30:30.1850
1877881,3601.0,2018-09-30 08:10:03.1790
1877882,3601.0,2018-09-30 12:20:13.6830


In [79]:
citybike[(citybike.start_hour.isin([18, 19, 20]))]['start station id'].value_counts().head()

False    1502902
True      374266
Name: starttime, dtype: int64

# 10. Выберите идентификаторы станций(end station id), в которые приезжают с 6 до 10 утра включительно

In [ ]:
end_stations = [3140, 3106, 3116, 369]

In [81]:
citybike[(citybike.end_hour.isin([6,7,8,9,10])) &
         (citybike['end station id'].isin(end_stations))]['end station id'].unique()

False    1450389
True      426779
Name: stoptime, dtype: int64